In [69]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [70]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy, qiskit
import circuit_cutter
import mlrecon_methods as ml
import json

import numpy as np
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, H_distance_dict, total_counts
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [72]:
with open('BeH2_auckland_nocut_orign_rerun.json') as f:
    all_list = json.load(f)
with open('BeH2_auckland_nocut_orign_rerun_mit.json') as f:
    all_mit_list = json.load(f)

In [10]:
len(all_list)

238

In [74]:
unmiti_list = all_list[-34:]
unmiti_mit_list = all_mit_list[-34:]
len(unmiti_list)

34

In [75]:
nocut_list = all_list[0:-34]
nocut_mit_list = all_mit_list[0:-34]
len(nocut_list)

204

In [105]:
len(all_mit_list)

34

In [76]:
# with open('BeH2_geneva_orign.json') as f:
#     unmiti_list = json.load(f)
with open('BeH2_auckland_check0.json') as f:
    miti_q0_list = json.load(f)
with open('BeH2_auckland_check1.json') as f:
    miti_q1_list = json.load(f)
with open('BeH2_auckland_check2.json') as f:
    miti_q2_list = json.load(f)
with open('BeH2_auckland_check3.json') as f:
    miti_q3_list = json.load(f)
with open('BeH2_auckland_check4.json') as f:
    miti_q4_list = json.load(f)
with open('BeH2_auckland_check5.json') as f:
    miti_q5_list = json.load(f)
with open('BeH2_auckland_check0_mit.json') as f:
    miti_q0_mit_list = json.load(f)
with open('BeH2_auckland_check1_mit.json') as f:
    miti_q1_mit_list = json.load(f)
with open('BeH2_auckland_check2_mit.json') as f:
    miti_q2_mit_list = json.load(f)
with open('BeH2_auckland_check3_mit.json') as f:
    miti_q3_mit_list = json.load(f)
with open('BeH2_auckland_check4_mit.json') as f:
    miti_q4_mit_list = json.load(f)
with open('BeH2_auckland_check5_mit.json') as f:
    miti_q5_mit_list = json.load(f)

In [77]:
len(nocut_list)

204

In [78]:
204/6

34.0

In [79]:
per_miti_results = []
for i in range(0, 6):
    temp_list = []
    for j in range(0, 34):
        filter_per_miti = norm_dict(filter_results(nocut_list[i * 34 + j], [0]))
        temp_list.append(filter_per_miti)
    per_miti_results.append(temp_list)

In [80]:
per_miti_results[0]

[{'000000': 0.01845018450184502,
  '000001': 0.013807880014284013,
  '010000': 0.019402452089037018,
  '010001': 0.013688846565885014,
  '010010': 0.01809308415664802,
  '010011': 0.013331746220688013,
  '010100': 0.02142602071182002,
  '010101': 0.015712415188668016,
  '010110': 0.017735983811451018,
  '010111': 0.012617545530294013,
  '011000': 0.016783716224259018,
  '011001': 0.011903344839900012,
  '011010': 0.017974050708249017,
  '011011': 0.013331746220688013,
  '011100': 0.01833115105344602,
  '011101': 0.016069515533865018,
  '011110': 0.017259850017855016,
  '011111': 0.013569813117486014,
  '000010': 0.01809308415664802,
  '100000': 0.018569217950244018,
  '100001': 0.01071301035591001,
  '100010': 0.02047375312462802,
  '100011': 0.015593381740269016,
  '100100': 0.01916438519223902,
  '100101': 0.013331746220688013,
  '100110': 0.02071182002142602,
  '100111': 0.015474348291870015,
  '101000': 0.015474348291870015,
  '101001': 0.012141411736698013,
  '101010': 0.016307582

Untimigated distribution

In [81]:
qubits = 6
# list of all possible measurement outcomes (bitstrings)
all_bits = [ "".join(bits) for bits in itertools.product(["0","1"], repeat = qubits) ]

In [82]:
def load_probs(miti_dist_list, bit_str):
    output_list = []
    for dist in miti_dist_list:
        try:
            miti_count = dist[bit_str]
        except:
            miti_count = 0
        output_list.append(miti_count)
    return output_list

In [83]:
def bit_weight(dist, index):
    #bitwise distribution
    weight_0 = 0
    weight_1 = 0
    for key in dist.keys():
        if key[len(key) - 1 - index] == '0':
            weight_0 += dist[key]
        elif key[len(key) - 1 - index] == '1':
            weight_1 += dist[key]
        else:
            print("Incorrect key value")
    return weight_0, weight_1

In [84]:
def update_dist(unmiti_dist, miti_dist, index):
    Ppost = {}
    w0, w1 = bit_weight(miti_dist, index)
    u_w0, u_w1 = bit_weight(unmiti_dist, index)
    if w0 == 0:
        w0 = 0.0000000000001
        w1 = 0.9999999999999
    if w1 == 0:
        w1 = 0.0000000000001
        w0 = 0.9999999999999
    if u_w0 == 0:
        u_w0 = 0.0000000000001
        u_w1 = 0.9999999999999
    if u_w1 == 0:
        u_w1 = 0.0000000000001
        u_w0 = 0.9999999999999
        
    for key in unmiti_dist.keys():
        if key[len(key) - 1 - index] == '0':
            Ppost[key] = unmiti_dist[key] / u_w0 * (w0)# / w1)
            #print(w0, w1, w0/w1, Ppost[key])
        elif key[len(key) - 1 - index] == '1':
            Ppost[key] = unmiti_dist[key] / u_w1 * (w1)# / w0)
            #print(w0, w1, w1/w0, Ppost[key])
        else:
            print("Incorrect key value")
    return Ppost

In [85]:
def combine_dist(orign_dist, dist_list):
    output_dist = {}
    for key in orign_dist:
        value = orign_dist[key]
        for dist in dist_list:
            value += dist[key]
        output_dist[key] = value
    return output_dist  

In [86]:
def bayesian_reconstruct(unmiti_dist, miti_dist_list, threshold = 0.0001):
    temp_dist = unmiti_dist.copy()
    h_dist = 1
    while h_dist > threshold:
        temp_dist_start = temp_dist.copy()
        ppost = [0] * len(miti_dist_list)
        for i in range(0, len(miti_dist_list)):
            ppost[i] = update_dist(temp_dist, miti_dist_list[i][0], miti_dist_list[i][1])
        temp_dist = combine_dist(temp_dist, ppost)
        temp_dist = norm_dict(temp_dist)
        h_dist = H_distance_dict(temp_dist, temp_dist_start)
        print("H-dist:", h_dist)
    return temp_dist

In [87]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [88]:
full_dist = []
for idx in range(0, len(per_miti_results[0])):
    miti_dist_index = [ [per_miti_results[0][idx],0],[per_miti_results[1][idx],1], [per_miti_results[2][idx],2], [per_miti_results[3][idx],3], [per_miti_results[4][idx],4], [per_miti_results[5][idx],5]]
    post_dist = bayesian_reconstruct(unmiti_list[idx], miti_dist_index)
    full_dist.append(post_dist)

H-dist: 70.00357133753646
H-dist: 0.01393861443516495
H-dist: 0.011865360966849826
H-dist: 0.01011925939613788
H-dist: 0.00864241945555635
H-dist: 0.007389360728611043
H-dist: 0.006323635418837053
H-dist: 0.005415582841444037
H-dist: 0.004640779518661337
H-dist: 0.003978932521232577
H-dist: 0.0034130647854299094
H-dist: 0.0029288988965399648
H-dist: 0.0025143798936463593
H-dist: 0.0021592982862456826
H-dist: 0.0018549872743917086
H-dist: 0.0015940762812884457
H-dist: 0.00137028816513381
H-dist: 0.0011782709567808585
H-dist: 0.0010134573245592698
H-dist: 0.0008719465984483207
H-dist: 0.0007504053425769399
H-dist: 0.000645983305567486
H-dist: 0.000556242203548649
H-dist: 0.0004790952663215375
H-dist: 0.0004127558463944437
H-dist: 0.0003556936823779466
H-dist: 0.00030659764234119096
H-dist: 0.0002643439629617795
H-dist: 0.0002279691564756597
H-dist: 0.00019664688670959946
H-dist: 0.00016966822317826194
H-dist: 0.0001464247724146333
H-dist: 0.00012639426151250165
H-dist: 0.0001091282127871

H-dist: 0.00016582737485786004
H-dist: 0.00014254458373575495
H-dist: 0.00012255926406532479
H-dist: 0.00010540024925926931
H-dist: 9.066431951057097e-05
H-dist: 70.00357133787925
H-dist: 0.030342100844245216
H-dist: 0.02200094939466841
H-dist: 0.017073647324877508
H-dist: 0.013678037503511584
H-dist: 0.011156745226549447
H-dist: 0.009203789765099234
H-dist: 0.007650819440239776
H-dist: 0.006394313355458978
H-dist: 0.0053654142253041605
H-dist: 0.004515623260964411
H-dist: 0.0038092994103602414
H-dist: 0.003219402630230975
H-dist: 0.0027249171019009155
H-dist: 0.0023092050491456645
H-dist: 0.0019589060893699627
H-dist: 0.00166317234143818
H-dist: 0.0014131191607932082
H-dist: 0.0012014196451548213
H-dist: 0.0010219982318260441
H-dist: 0.0008697946209049259
H-dist: 0.0007405789093058217
H-dist: 0.0006308048627134206
H-dist: 0.0005374921479846721
H-dist: 0.0004581309287781826
H-dist: 0.0003906039798387456
H-dist: 0.0003331226939680732
H-dist: 0.00028417422152477047
H-dist: 0.000242477609

In [89]:
pauli_list = read_from_file('BeH2_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [90]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(full_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [91]:
#results without cut
final_expect_val

(-3.3611878651503235+0j)

In [92]:
#with cutting
full_dist = []
for idx in range(0, len(per_miti_results[0])):
    miti_dist_index = [ [miti_q0_list[idx],0],[miti_q1_list[idx],1], [miti_q2_list[idx],2], [miti_q3_list[idx],3], [miti_q4_list[idx],4], [miti_q5_list[idx],5]]
    post_dist = bayesian_reconstruct(unmiti_list[idx], miti_dist_index)
    full_dist.append(post_dist)

H-dist: 70.00357133755685
H-dist: 0.015871136139910567
H-dist: 0.01333155985700341
H-dist: 0.011217285605807691
H-dist: 0.00944970759355526
H-dist: 0.007967596546195974
H-dist: 0.00672222791304993
H-dist: 0.005674203894350625
H-dist: 0.004791289637509355
H-dist: 0.004046881364600509
H-dist: 0.0034188834441696374
H-dist: 0.002888859989875847
H-dist: 0.002441377405563951
H-dist: 0.00206348428261321
H-dist: 0.001744293189900968
H-dist: 0.0014746401338133791
H-dist: 0.0012468046051696878
H-dist: 0.001054277787885886
H-dist: 0.0008915696304269244
H-dist: 0.0007540476425122214
H-dist: 0.0006378018195815053
H-dist: 0.0005395312273999993
H-dist: 0.0004564486308808864
H-dist: 0.0003862002087748107
H-dist: 0.0003267979139549363
H-dist: 0.00027656245401399805
H-dist: 0.00023407520366704337
H-dist: 0.00019813763653809936
H-dist: 0.0001677370919713033
H-dist: 0.00014201788197886844
H-dist: 0.00012025690151629744
H-dist: 0.00010184303757018719
H-dist: 8.625978351106582e-05
H-dist: 70.00357133792839


H-dist: 0.0006876339859750579
H-dist: 0.0006318069638730863
H-dist: 0.0005808408284843781
H-dist: 0.0005342643738900568
H-dist: 0.0004916591421318222
H-dist: 0.00045265246018169256
H-dist: 0.00041691155797124793
H-dist: 0.00038413857450194007
H-dist: 0.00035406629731651504
H-dist: 0.0003264545107658905
H-dist: 0.00030108685232484064
H-dist: 0.0002777680950755959
H-dist: 0.0002563217894711723
H-dist: 0.0002365882094522904
H-dist: 0.00021842255757587248
H-dist: 0.00020169339152897997
H-dist: 0.00018628124064484292
H-dist: 0.00017207738611384971
H-dist: 0.0001589827827329706
H-dist: 0.00014690710344795958
H-dist: 0.00013576789076052355
H-dist: 0.00012548980141283478
H-dist: 0.00011600393271447034
H-dist: 0.00010724722051374611
H-dist: 9.916190019441144e-05
H-dist: 70.00357133797284
H-dist: 0.037480583129571254
H-dist: 0.03123220067181139
H-dist: 0.02633153983878965
H-dist: 0.02238533503189861
H-dist: 0.01914912262617828
H-dist: 0.016460407944296616
H-dist: 0.014205348160976804
H-dist: 0.0

H-dist: 0.0005429211514218986
H-dist: 0.0005026380624328745
H-dist: 0.0004653452052859894
H-dist: 0.00043082031388792867
H-dist: 0.000398857710259311
H-dist: 0.00036926705155418713
H-dist: 0.00034187217507592095
H-dist: 0.00031651003288135524
H-dist: 0.0002930297084462978
H-dist: 0.00027129150862290324
H-dist: 0.00025116612477569584
H-dist: 0.00023253385755562242
H-dist: 0.0002152839002747377
H-dist: 0.00019931367628915964
H-dist: 0.00018452822619379093
H-dist: 0.00017083964098641997
H-dist: 0.00015816653767687877
H-dist: 0.00014643357410407386
H-dist: 0.00013557099998368348
H-dist: 0.0001255142414455588
H-dist: 0.00011620351653513826
H-dist: 0.000107583479349786
H-dist: 9.960289066091766e-05
H-dist: 70.00357133751332
H-dist: 0.011626211080345797
H-dist: 0.010447480380720468
H-dist: 0.00942070458438014
H-dist: 0.008521553220162747
H-dist: 0.00773015669805861
H-dist: 0.00703021298856882
H-dist: 0.006408293399602612
H-dist: 0.005853294946224977
H-dist: 0.005356003703353611
H-dist: 0.0049

In [93]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(full_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [94]:
final_expect_val

(-3.4288596525989172+0j)

In [95]:
#no bayesian recombine:
miti_dist_list = [miti_q0_list, miti_q1_list, miti_q2_list, miti_q3_list, miti_q4_list, miti_q5_list]
total = 0
for idx in range(0, 6):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(miti_dist_list[idx][i], shots = 1, Pauli = Pauli_tuple[0])  
    total += final_expect_val
    print("expectation value for mitigating qubit{}:{}".format(idx, final_expect_val) )

expectation value for mitigating qubit0:(-3.386702477271455+0j)
expectation value for mitigating qubit1:(-3.376942626397123+0j)
expectation value for mitigating qubit2:(-3.3938414179594116+0j)
expectation value for mitigating qubit3:(-3.3581690081275855+0j)
expectation value for mitigating qubit4:(-3.34132956096899+0j)
expectation value for mitigating qubit5:(-3.3103598298269437+0j)


In [96]:
total/6

(-3.361224153425251+0j)

In [97]:
#no cutting
total = 0
for idx in range(0, 6):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(per_miti_results[idx][i], shots = 1, Pauli = Pauli_tuple[0])  
    total += final_expect_val
    print("expectation value for mitigating qubit{}:{}".format(idx, final_expect_val) )

expectation value for mitigating qubit0:(-3.3359091103197245+0j)
expectation value for mitigating qubit1:(-3.3644912144954064+0j)
expectation value for mitigating qubit2:(-3.3516225532492903+0j)
expectation value for mitigating qubit3:(-3.3131583890829823+0j)
expectation value for mitigating qubit4:(-3.2514631009752333+0j)
expectation value for mitigating qubit5:(-3.3198281138693635+0j)


In [98]:
total/6

(-3.322745413665334+0j)

In [99]:
#untimigated result:
unmiti_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        unmiti_expect_val += coeff * evaluation(unmiti_list[i], shots = total_counts(unmiti_list[i]), Pauli = Pauli_tuple[0])

In [100]:
unmiti_expect_val

(-3.3380599474310233+0j)

Experiments with mthree

In [101]:
#untimigated result:
unmiti_mit_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        unmiti_mit_expect_val += coeff * evaluation(unmiti_mit_list[i], shots = total_counts(unmiti_mit_list[i]), Pauli = Pauli_tuple[0])

In [102]:
unmiti_mit_expect_val

(-3.399392338014033+0j)

In [104]:
len(nocut_mit_list)

0

In [106]:
#no bayesian recombine:
miti_mit_dist_list = [miti_q0_mit_list, miti_q1_mit_list, miti_q2_mit_list, miti_q3_mit_list, miti_q4_mit_list, miti_q5_mit_list]
total = 0
for idx in range(0, 6):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(miti_mit_dist_list[idx][i], shots = 1, Pauli = Pauli_tuple[0])  
    total += final_expect_val
    print("expectation value for mitigating qubit{}:{}".format(idx, final_expect_val) )

expectation value for mitigating qubit0:(-3.4205953293403186+0j)
expectation value for mitigating qubit1:(-3.4273167587829105+0j)
expectation value for mitigating qubit2:(-3.4308676970255987+0j)
expectation value for mitigating qubit3:(-3.401242775393795+0j)
expectation value for mitigating qubit4:(-3.3837601040853755+0j)
expectation value for mitigating qubit5:(-3.3499784566899775+0j)


In [107]:
#with cutting
full_mit_dist = []
for idx in range(0, len(per_miti_results[0])):
    miti_mit_dist_index = [ [miti_q0_mit_list[idx],0],[miti_q1_mit_list[idx],1], [miti_q2_mit_list[idx],2], [miti_q3_mit_list[idx],3], [miti_q4_mit_list[idx],4], [miti_q5_mit_list[idx],5]]
    post_dist = bayesian_reconstruct(unmiti_mit_list[idx], miti_mit_dist_index)
    full_mit_dist.append(post_dist)

H-dist: 70.0035713375712
H-dist: 0.017090819163891807
H-dist: 0.014296212529463302
H-dist: 0.011982778081734074
H-dist: 0.010058195257230498
H-dist: 0.00845152767243052
H-dist: 0.007106938998635244
H-dist: 0.005979680361859204
H-dist: 0.0050334084305134605
H-dist: 0.004238317101884157
H-dist: 0.0035697869536683
H-dist: 0.003007377102379412
H-dist: 0.0025340520293860138
H-dist: 0.0021355753850754214
H-dist: 0.001800026322930746
H-dist: 0.0015174083328895489
H-dist: 0.0012793296085269951
H-dist: 0.0010787398415666358
H-dist: 0.000909712235682395
H-dist: 0.0007672622052545944
H-dist: 0.0006471961153435808
H-dist: 0.0005459847961521383
H-dist: 0.00046065759602009526
H-dist: 0.0003887135274947405
H-dist: 0.0003280466798410766
H-dist: 0.00027688356398449936
H-dist: 0.0002337304533346843
H-dist: 0.00019732910793982024
H-dist: 0.0001666195356513038
H-dist: 0.00014070866404258432
H-dist: 0.00011884397954553883
H-dist: 0.00010039134248296212
H-dist: 8.481631379466699e-05
H-dist: nan
H-dist: nan


/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation/utils/utils.py:57: RuntimeWarning: invalid value encountered in sqrt
  sum += (np.sqrt(p[key]) - np.sqrt(q[key]))**2


In [108]:
final_mit_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_mit_expect_val += coeff * evaluation(full_mit_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [109]:
final_mit_expect_val

(-3.3983686038499994+0j)